## Harjoitustyö



In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


ModuleNotFoundError: No module named 'scrapy'

In [3]:
!pip install scrapy

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 254 kB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 251 kB 15.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 18.9 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 358 kB/s  eta 0:00:01
     |████████████████████████████████| 3.1 MB 58.8 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 55.9 MB/s eta 0:00:01
     |████████████████████████████████| 74 kB 297 kB/s  eta 0:00:01
  Created wheel for protego: filename=Protego-0.1.16-py3-none-any.whl size=7765 sha256=dd8e62d057870dd372f725885

In [4]:
!scrapy genspider etuovi_scraper etuovi.com

Created spider 'etuovi_scraper' using template 'basic' 


In [5]:
# -*- coding: utf-8 -*-
import scrapy
# raapija, joka kerää etuovi.com:sta ilmoituksien tiedoista
# asuntotyypin, osoitteen, hinnan, valmistuvuoden ja pinta-alan.
# (Kyseessä Tampereen omakotitalot)
class EtuoviScraperSpider(scrapy.Spider):
    name = 'etuovi_scraper'
    allowed_domains = ['etuovi.com']
    # assing a product-review-page url below
    start_urls = ['https://www.etuovi.com/myytavat-asunnot/tampere?haku=M1626860813&sivu=0']

  # Funkito parse(self, response) käy läpi aloitusosoitteesta eteenpäin
  # sivuja, joilta dataa kerätään. Parse kutsuu parse_page-funktiota, jossa
  # on määritelty varsinainen kerättävä tieto.
  # max 5 sivua tässä esimerkissä
    def parse(self, response):
        for i in range(1, 6):
            url=response.request.url[:-1]+str(i)
            yield scrapy.Request(url, callback=self.parse_page)
        
    
    def parse_page(self, response):
        description_texts = response.css('[class="flexboxgrid__col-xs-12__1I1LS"] *::text').extract()
        size_text = response.css('[title="Asuinpinta-ala / kokonaispinta-ala"]::text').extract()
        year_text = response.css('[class="flexboxgrid__col-xs-3__3Kf8r flexboxgrid__col-md-4__2DYW-"] > span::text').extract()
        price_text = response.css('[class="flexboxgrid__col-xs-4__p2Lev flexboxgrid__col-md-4__2DYW-"] > span::text').extract()

        for i in range(len(price_text)):
            
            yield {
                'house_type' : description_texts[3*i+1],
                'address': description_texts[3*i+3],
                'price': price_text[i],
                'year': year_text[i],
                'size': size_text[i]
            }

In [ ]:
price_text[i]=price_text[i].replace("\xa0", "")
            price_text[i]=price_text[i].replace("\u20ac", "")

In [6]:
!scrapy runspider etuovi_scraper.py -o out.json

2021-04-21 08:24:35 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-04-21 08:24:35 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.7.10 (default, Feb 26 2021, 18:47:35) - [GCC 7.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.4.7, Platform Linux-4.14.225-169.362.amzn2.x86_64-x86_64-with-debian-10.6
2021-04-21 08:24:35 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-04-21 08:24:35 [scrapy.crawler] INFO: Overridden settings:
{'SPIDER_LOADER_WARN_ONLY': True}
2021-04-21 08:24:35 [scrapy.extensions.telnet] INFO: Telnet Password: e589175ec2bd7a56
2021-04-21 08:24:35 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
202